In [3]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from src.data.load_data import load_raw_data
from src.data.data_profiler import check_data
import polars as pl

df_January = load_raw_data(year=2025, month=1)
df_February = load_raw_data(year=2025, month=2)
df_March = load_raw_data(year=2025, month=3)
df_April = load_raw_data(year=2025, month=4)
df_May = load_raw_data(year=2025,month=5)
df_June  = load_raw_data(year=2025, month=6)
df_July = load_raw_data(year=2025, month=7)
df_August = load_raw_data(year=2025, month=8)
df_September  = load_raw_data(year=2025, month=9)
df_October  = load_raw_data(year=2025, month=10)

In [5]:
dfs = [
    df_January, df_February, df_March, df_April, df_May, df_June,
    df_July, df_August, df_September, df_October
]

df_full = pl.concat(dfs)

In [6]:
check_data(df_full)



╔══════════════════════════════════════════════════════════╗
║               DATA PROFILING REPORT                      ║
╚══════════════════════════════════════════════════════════╝


  БАЗОВАЯ ИНФОРМАЦИЯ

Размер датасета:
  Строк: 5,055,832
  Колонок: 13
  Размер в памяти: 638.30 MB

Типы данных:
  Datetime(time_unit='us', time_zone=None): 2 колонок
  Float64: 4 колонок
  String: 7 колонок

Колонки:
   1. ride_id                        (String)
   2. rideable_type                  (String)
   3. started_at                     (Datetime(time_unit='us', time_zone=None))
   4. ended_at                       (Datetime(time_unit='us', time_zone=None))
   5. start_station_name             (String)
   6. start_station_id               (String)
   7. end_station_name               (String)
   8. end_station_id                 (String)
   9. start_lat                      (Float64)
  10. start_lng                      (Float64)
  11. end_lat                        (Float64)
  12. end_lng   

In [8]:
import polars as pl

def clean_rides_data(df: pl.DataFrame) -> pl.DataFrame:
    """
    Основная функция очистки данных о поездках
    """

    df_clean = df.clone()
    
    print("Начинаем очистку ...")
    print(f"Исходный размер: {df_clean.shape}")
    
    
    # Убедимся, что временные метки в правильном формате
    df_clean = df_clean.with_columns([
        pl.col("started_at").cast(pl.Datetime(time_unit="us")),
        pl.col("ended_at").cast(pl.Datetime(time_unit="us"))
    ])
    
    CHICAGO_LAT_BOUNDS = (41.6, 42.2)
    CHICAGO_LNG_BOUNDS = (-88.0, -87.5)
    
    coord_condition = (
        pl.col("start_lat").is_between(*CHICAGO_LAT_BOUNDS) &
        pl.col("start_lng").is_between(*CHICAGO_LNG_BOUNDS) &
        pl.col("end_lat").is_between(*CHICAGO_LAT_BOUNDS) &
        pl.col("end_lng").is_between(*CHICAGO_LNG_BOUNDS)
    )
    
    zero_coord_condition = (
        (pl.col("end_lat") != 0) & 
        (pl.col("end_lng") != 0)
    )
    
    df_clean = df_clean.filter(coord_condition & zero_coord_condition)
    print(f"После фильтрации координат: {df_clean.shape}")
    
    # длительность в минутах
    df_clean = df_clean.with_columns([
        ((pl.col("ended_at") - pl.col("started_at")).dt.total_seconds() / 60).alias("duration_minutes")
    ])
    
    time_condition = (
        (pl.col("duration_minutes") > 0) &  # Длительность положительная
        (pl.col("duration_minutes") <= 24 * 60) &  # Не более суток
        (pl.col("ended_at") >= pl.col("started_at"))  # Конец после начала
    )
    
    df_clean = df_clean.filter(time_condition & (pl.col("duration_minutes") >= 1))
    print(f"После фильтрации по времени: {df_clean.shape}")
    
    missing_analysis = df_clean.group_by("rideable_type").agg([
        pl.col("start_station_name").is_null().mean().alias("pct_missing_start_station"),
        pl.col("end_station_name").is_null().mean().alias("pct_missing_end_station"),
        pl.count().alias("count")
    ]).sort("count", descending=True)
    
    print("\nАнализ пропусков по типам самокатов:")
    print(missing_analysis)
    
    # Обычно electric bikes могут не иметь станций (бесдоковые)
    # Но для классических bikes станции должны быть
    
    #флаги для поездок без станций
    df_clean = df_clean.with_columns([
        pl.col("start_station_name").is_null().alias("missing_start_station"),
        pl.col("end_station_name").is_null().alias("missing_end_station")
    ])
    
    
    # Проверяем уникальность ride_id
    duplicate_ids = df_clean.group_by("ride_id").agg(pl.count().alias("count")).filter(pl.col("count") > 1)
    if duplicate_ids.height > 0:
        print(f"\nНайдено {duplicate_ids.height} дубликатов ride_id")
        # Оставляем первую запись для каждого ride_id
        df_clean = df_clean.unique(subset=["ride_id"], keep="first")

    valid_rideable_types = ["electric_bike", "classic_bike", "docked_bike"]
    valid_member_types = ["member", "casual"]
    
    df_clean = df_clean.filter(
        pl.col("rideable_type").is_in(valid_rideable_types) &
        pl.col("member_casual").is_in(valid_member_types)
    )
    
    # Поездки без конечной станции И без конечных координат
    potential_issues = df_clean.filter(
        pl.col("missing_end_station") & 
        ((pl.col("end_lat").is_null()) | (pl.col("end_lng").is_null()))
    )
    
    print(f"\nПотенциальные проблемные поездки (без конечной точки): {potential_issues.height}")
    
    # Поездки с очень большой длительностью
    long_rides = df_clean.filter(pl.col("duration_minutes") > 12 * 60)  # Более 12 часов
    print(f"Поездки длительностью > 12 часов: {long_rides.height}")

    # Добавляем день недели, час и месяц для анализа
    df_clean = df_clean.with_columns([
        pl.col("started_at").dt.weekday().alias("day_of_week"),
        pl.col("started_at").dt.hour().alias("hour_of_day"),
        pl.col("started_at").dt.month().alias("month"),
        pl.col("started_at").dt.date().alias("date")
    ])
    
    df_clean_version = df_clean.filter(
        ~pl.col("missing_start_station") & 
        ~pl.col("missing_end_station")
    )
    
    print(f"\nФинальные размеры:")
    print(f"- Полный очищенный набор: {df_clean.shape}")
    print(f"- Только поездки со станциями: {df_clean_version.shape}")
    
    # отчет об очистке
    report = {
        "initial_rows": df.height,
        "final_rows": df_clean.height,
        "rows_removed": df.height - df_clean.height,
        "pct_removed": round((1 - df_clean.height / df.height) * 100, 2),
        "missing_start_station_pct": round(df_clean["missing_start_station"].mean() * 100, 2),
        "missing_end_station_pct": round(df_clean["missing_end_station"].mean() * 100, 2),
        "avg_duration_minutes": round(df_clean["duration_minutes"].mean(), 2),
        "median_duration_minutes": round(df_clean["duration_minutes"].median(), 2)
    }
    
    print("\nОтчет об очистке:")
    for key, value in report.items():
        print(f"  {key}: {value}")
    
    return df_clean, df_clean_version, report

df_clean, df_clean_with_stations, report = clean_rides_data(df_full)

# Доп анализ аномалий
def analyze_anomalies(df: pl.DataFrame):
    """
    Анализ потенциальных аномалий и краж
    """
    print("\n" + "="*50)
    print("АНАЛИЗ АНОМАЛИЙ")
    print("="*50)
    
    # Поездки без начальной станции
    no_start_station = df.filter(pl.col("missing_start_station"))
    print(f"1. Поездки без начальной станции: {no_start_station.height}")
    print("   Распределение по типам самокатов:")
    print(no_start_station.group_by("rideable_type").agg(pl.count()).sort("count", descending=True))
    
    # Поездки без конечной станции
    no_end_station = df.filter(pl.col("missing_end_station"))
    print(f"\n2. Поездки без конечной станции: {no_end_station.height}")
    print("   Распределение по типам пользователей:")
    print(no_end_station.group_by("member_casual").agg(pl.count()).sort("count", descending=True))
    
    # Поездки с очень короткой/длинной длительностью
    print(f"\n3. Анализ длительности поездок:")
    print(f"   Менее 5 минут: {df.filter(pl.col('duration_minutes') < 5).height}")
    print(f"   Более 3 часов: {df.filter(pl.col('duration_minutes') > 180).height}")
    
    # Проверка круглосуточных поездок
    overnight_rides = df.filter(
        (pl.col("duration_minutes") > 60 * 6) &  # Более 6 часов
        (pl.col("started_at").dt.hour() >= 18) &  # Начало вечером
        (pl.col("ended_at").dt.hour() <= 8)       # Окончание утром
    )
    print(f"\n4. Ночные длительные поездки (>6 часов): {overnight_rides.height}")
    
    # Анализ по дням недели
    print(f"\n5. Распределение поездок по дням недели:")
    day_of_week_stats = df.group_by("day_of_week").agg([
        pl.count().alias("count"),
        pl.col("duration_minutes").median().alias("median_duration")
    ]).sort("day_of_week")
    print(day_of_week_stats)
    
    return {
        "no_start_station": no_start_station,
        "no_end_station": no_end_station,
        "overnight_rides": overnight_rides
    }

# Запускаем анализ
anomalies = analyze_anomalies(df_clean)

# Сохранение очищенных данных
def save_cleaned_data(df_full_clean: pl.DataFrame, df_with_stations: pl.DataFrame):
    """
    Сохранение очищенных данных
    """
    df_full_clean.write_parquet("data/processed/rides_2025_cleaned.parquet")
    df_with_stations.write_parquet("data/processed/rides_2025_with_stations.parquet")   
    df_with_stations.write_csv("data/processed/rides_2025_with_stations_sample.csv")
    
    print("\nДанные сохранены:")
    print("- rides_2025_cleaned.parquet (все очищенные данные)")
    print("- rides_2025_with_stations.parquet (только со станциями)")
    print("- rides_2025_with_stations_sample.csv (выборка для просмотра)")


save_cleaned_data(df_clean, df_clean_with_stations)

Начинаем очистку ...
Исходный размер: (5055832, 13)
После фильтрации координат: (5050770, 13)
После фильтрации по времени: (4916696, 14)

Анализ пропусков по типам самокатов:
shape: (2, 4)
┌───────────────┬───────────────────────────┬─────────────────────────┬─────────┐
│ rideable_type ┆ pct_missing_start_station ┆ pct_missing_end_station ┆ count   │
│ ---           ┆ ---                       ┆ ---                     ┆ ---     │
│ str           ┆ f64                       ┆ f64                     ┆ u32     │
╞═══════════════╪═══════════════════════════╪═════════════════════════╪═════════╡
│ electric_bike ┆ 0.319215                  ┆ 0.326982                ┆ 3134216 │
│ classic_bike  ┆ 0.0                       ┆ 0.000003                ┆ 1782480 │
└───────────────┴───────────────────────────┴─────────────────────────┴─────────┘


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6887/2353456445.py:55: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count")
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6887/2353456445.py:72: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  duplicate_ids = df_clean.group_by("ride_id").agg(pl.count().alias("count")).filter(pl.col("count") > 1)



Потенциальные проблемные поездки (без конечной точки): 0
Поездки длительностью > 12 часов: 1849

Финальные размеры:
- Полный очищенный набор: (4916696, 20)
- Только поездки со станциями: (3333377, 20)

Отчет об очистке:
  initial_rows: 5055832
  final_rows: 4916696
  rows_removed: 139136
  pct_removed: 2.75
  missing_start_station_pct: 20.35
  missing_end_station_pct: 20.84
  avg_duration_minutes: 15.14
  median_duration_minutes: 9.87

АНАЛИЗ АНОМАЛИЙ
1. Поездки без начальной станции: 1000489
   Распределение по типам самокатов:
shape: (1, 2)
┌───────────────┬─────────┐
│ rideable_type ┆ count   │
│ ---           ┆ ---     │
│ str           ┆ u32     │
╞═══════════════╪═════════╡
│ electric_bike ┆ 1000489 │
└───────────────┴─────────┘


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6887/2353456445.py:148: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print(no_start_station.group_by("rideable_type").agg(pl.count()).sort("count", descending=True))
/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6887/2353456445.py:154: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  print(no_end_station.group_by("member_casual").agg(pl.count()).sort("count", descending=True))



2. Поездки без конечной станции: 1024838
   Распределение по типам пользователей:
shape: (2, 2)
┌───────────────┬────────┐
│ member_casual ┆ count  │
│ ---           ┆ ---    │
│ str           ┆ u32    │
╞═══════════════╪════════╡
│ member        ┆ 636118 │
│ casual        ┆ 388720 │
└───────────────┴────────┘

3. Анализ длительности поездок:
   Менее 5 минут: 959510
   Более 3 часов: 9408

4. Ночные длительные поездки (>6 часов): 411

5. Распределение поездок по дням недели:


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_6887/2353456445.py:172: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count"),


shape: (7, 3)
┌─────────────┬────────┬─────────────────┐
│ day_of_week ┆ count  ┆ median_duration │
│ ---         ┆ ---    ┆ ---             │
│ i8          ┆ u32    ┆ f64             │
╞═════════════╪════════╪═════════════════╡
│ 1           ┆ 648423 ┆ 9.333333        │
│ 2           ┆ 694505 ┆ 9.266667        │
│ 3           ┆ 676332 ┆ 9.116667        │
│ 4           ┆ 743747 ┆ 9.283333        │
│ 5           ┆ 753021 ┆ 9.8             │
│ 6           ┆ 760315 ┆ 11.416667       │
│ 7           ┆ 640353 ┆ 11.3            │
└─────────────┴────────┴─────────────────┘

Данные сохранены:
- rides_2025_cleaned.parquet (все очищенные данные)
- rides_2025_with_stations.parquet (только со станциями)
- rides_2025_with_stations_sample.csv (выборка для просмотра)
